<a href="https://colab.research.google.com/github/MrKozelberg/fcc_predict_health_costs_with_regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

### Convertation categorical data to numbers

In [ ]:
for name, column in dataset.items():
  if column.dtype == object:
    unique_values = dataset[name].unique()

    dataset[name].replace(unique_values, 
                          np.arange(len(unique_values)),
                          inplace=True)

In [ ]:
dataset.tail()

### 'Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.'

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=1)
test_dataset = dataset.drop(train_dataset.index)

### 'pop off the "expenses" column from these datasets to create new datasets called train_labels and test_labels.'

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

### Normalization

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

normalizer.adapt(np.array(train_dataset))

### Building the model

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(100),
    layers.Dense(50, activation='relu'),
    layers.Dense(10),
    layers.Dense(1)
])

# Configuration the model with Keras Model.compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01),
    loss="mse", metrics=["mae", "mse"])

# Training the model with Model.fit
history = model.fit(
    train_dataset,
    train_labels,
    batch_size=32,
    epochs=30,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
plt.plot(np.arange(30), history.history['mae'])
plt.xlim(0,30)
plt.axhline(y=3500, linestyle='--', color='black')

print(history.history['mae'][-1])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
